In [212]:
import fasttext
import pandas as pd
import numpy as np
import csv
from helpers import *
from textblob import Word, TextBlob

In [213]:
data_full = read_file("file/train_pos_full.txt", "file/train_neg_full.txt")

b'Skipping line 760409: expected 1 fields, saw 2\nSkipping line 790939: expected 1 fields, saw 2\n'


In [214]:
# fitting data into the form required by fasttext
data_raw_full = gen_raw_data(data_full)

In [215]:
# removing punctuation
data_punc_full = data_raw_full.copy()
data_punc_full['text'] = data_punc_full['text'].str.replace('[^\w\s]','')

# removing digits
data_dig_full = data_punc_full.copy()
data_dig_full['text'] = data_dig_full['text'].str.replace('\d+','')

# removing commom words
data_stop_full = data_dig_full.copy()
freq_full = pd.Series(' '.join(data_stop_full['text']).split()).value_counts()
freq = freq_full[:10]
freq = list(freq.index)
data_stop_full['text'] = data_stop_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [216]:
# write to csv for training
data_raw_full.to_csv('file/data_raw.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
data_punc_full.to_csv('file/data_punc.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
data_dig_full.to_csv('file/data_dig.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
data_stop_full.to_csv('file/data_stop.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [219]:
model = fasttext.train_supervised(input="file/data_raw.txt",
                                  lr=0.01,
                                  epoch=5,
                                  wordNgrams=2,
                                  ws=5,
                                  dim=200,
                                  minCount=1,
                                  loss='softmax')

#lr, standard range
#epoch, standard range
#wordNgrams, standard range
#ws, window size
#dim, size of word vectors
#minCount, minimal number of word occurences
#loss, loss function {ns, hs, softmax, ova}

In [220]:
cv(4, data_raw_full)

KeyboardInterrupt: 

## Generate submission

In [221]:
# read file
data_test = pd.read_table("file/test_data.txt", header = None)

In [222]:
# adjust to fasttext
data_test.columns = ['text']
data_test['text'] = data_test['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

# drop the id read into the text
data_test['text'] = data_test['text'].replace('^([^,]+),',' ', regex=True)

In [223]:
# drop punctuation
data_test['text'] = data_test['text'].str.replace('[^\w\s]','')

In [224]:
# drop digits
data_test['text'] = data_test['text'].str.replace('\d+','')

In [225]:
# drop commom words
freq_test = pd.Series(' '.join(data_test['text']).split()).value_counts()
freq_test = freq_test[:10]
freq_test = list(freq_test.index)
data_test['text'] = data_test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_test))

In [226]:
data_test

,text
0,sea doo pro sea scooter sports with portable s...
1,shucks well work all week so now cant come che...
2,cant stay away from bug thats baby
3,no maam lol im perfectly fine not contagious a...
4,whenever fall asleep watching tv always wake u...
...,...
9995,had nice time w friend lastnite
9996,no its not please stop
9997,not without daughter dvd twotime oscar r winne...
9998,have fun in class sweetcheeks


In [146]:
gen_sub()